# FedAvg (clientes com dados completos)

In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision.transforms as transforms
import torchvision.datasets as datasets
from torch.utils.data import DataLoader, Subset
import copy
import torchvision.models as models
import random

random.seed(123)
torch.manual_seed(123)
torch.cuda.manual_seed(123)

# Configuração do dispositivo
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Função para selecionar o modelo sem pré-treinamento
def select_model(architecture):
    if architecture == 'alexnet':
        model = models.alexnet(weights=None)
        # model = models.alexnet(weights=models.AlexNet_Weights.IMAGENET1K_V1)
        model.classifier[6] = nn.Linear(model.classifier[6].in_features, 10)
    elif architecture == 'vgg11':
        model = models.vgg11(weights=None)
        # model = models.vgg11(weights=models.VGG11_Weights.IMAGENET1K_V1)
        model.classifier[6] = nn.Linear(model.classifier[6].in_features, 10)
    elif architecture == 'resnet18':
        model = models.resnet18(weights=None)
        # model = models.resnet18(weights=models.ResNet18_Weights.IMAGENET1K_V1)
        model.fc = nn.Linear(model.fc.in_features, 10)
    elif architecture == 'mobilenet_v2':
        model = models.mobilenet_v2(weights=None)
        # model = models.mobilenet_v2(weights=models.MobileNet_V2_Weights.IMAGENET1K_V1)
        model.classifier[1] = nn.Linear(model.classifier[1].in_features, 10)
    elif architecture == 'squeezenet':
        model = models.squeezenet1_1(weights=None)
        # model = models.squeezenet1_1(weights=models.SqueezeNet1_1_Weights.IMAGENET1K_V1)
        model.classifier[1] = nn.Conv2d(512, 10, kernel_size=(1, 1), stride=(1, 1))
        model.num_classes = 10
    else:
        raise ValueError("Arquitetura não suportada: escolha 'alexnet', 'vgg11', 'resnet18', 'mobilenet_v2' ou 'squeezenet'")

    return model.to(device)
# Função para criação do dataset para cada cliente
def create_subset(dataset, subset_size):
    indices = list(range(len(dataset)))
    subset_indices = random.sample(indices, subset_size)
    return Subset(dataset, subset_indices)

# Classe para o cliente que faz treinamento local
class Client:
    def __init__(self, model, dataloader, device):
        self.model = copy.deepcopy(model).to(device)
        self.dataloader = dataloader
        self.optimizer = optim.Adam(self.model.parameters(), lr=0.0001, weight_decay=1e-5)
        self.device = device

    def local_train(self, criterion, num_epochs=1):
        self.model.train()
        for epoch in range(num_epochs):
            for images, labels in self.dataloader:
                images, labels = images.to(self.device), labels.to(self.device)
                self.optimizer.zero_grad()
                outputs = self.model(images)
                loss = criterion(outputs, labels)
                loss.backward()
                self.optimizer.step()

    def get_parameters(self):
        return {name: param.clone().detach() for name, param in self.model.state_dict().items()}

    def set_parameters(self, global_parameters):
        self.model.load_state_dict(global_parameters)

# Função principal de treinamento federado usando FedAvg
def federated_training_fedavg(architecture, num_clients, num_rounds):
    # Inicializar modelo base e critério de perda
    base_model = select_model(architecture)
    criterion = nn.CrossEntropyLoss()

    # Criar DataLoaders para cada cliente
    trainset = datasets.CIFAR10(root='./data', train=True, download=True, transform=transform)
    trainloaders = [DataLoader(create_subset(trainset, 50000), batch_size=64, shuffle=True) for _ in range(num_clients)]

    testset = datasets.CIFAR10(root='./data', train=False, download=True, transform=transform)
    testloader = DataLoader(create_subset(testset, 10000), batch_size=64, shuffle=False)

    # Inicializar clientes com dados locais
    clients = [Client(base_model, trainloaders[i], device) for i in range(num_clients)]
    global_model = copy.deepcopy(base_model)

    # Treinamento federado com FedAvg
    for round_num in range(num_rounds):
        client_models = []

        # Treinamento local em cada cliente
        for client in clients:
            client.set_parameters(global_model.state_dict())
            client.local_train(criterion, num_epochs=5)
            client_models.append(client.get_parameters())

        # Agregação FedAvg
        new_global_parameters = {}
        for name in client_models[0].keys():
            #new_global_parameters[name] = torch.mean(torch.stack([client[name] for client in client_models]), dim=0)
            new_global_parameters[name] = torch.mean(torch.stack([client[name].float() for client in client_models]), dim=0)

        # Atualizar modelo global
        global_model.load_state_dict(new_global_parameters)

        # Avaliação do modelo global
        test_loss, test_accuracy = evaluate_model(global_model, testloader, criterion)
        print(f'Round {round_num + 1}/{num_rounds}, Test Loss: {test_loss:.4f}, Test Accuracy: {test_accuracy:.4f}')

    # Avaliação final no conjunto de teste
    print("Treinamento FedAvg concluído.")
    test_loss, test_accuracy = evaluate_model(global_model, testloader, criterion)
    print(f'Avaliação final no conjunto de teste - Loss: {test_loss:.4f}, Accuracy: {test_accuracy:.4f}')

# Função de avaliação do modelo
def evaluate_model(model, dataloader, criterion):
    model.eval()
    total_loss = 0
    correct = 0
    total = 0
    with torch.no_grad():
        for images, labels in dataloader:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            loss = criterion(outputs, labels)
            total_loss += loss.item()
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    avg_loss = total_loss / len(dataloader)
    accuracy = correct / total
    return avg_loss, accuracy

# Parâmetros de execução
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
])

In [2]:
num_clients = 10
num_rounds = 30

## SqueeNet

In [8]:
architecture = 'squeezenet' #'resnet18', 'mobilenet_v2', 'squeezenet' ou 'alexnet'
federated_training_fedavg(architecture, num_clients, num_rounds)

Files already downloaded and verified
Files already downloaded and verified
Round 1/30, Test Loss: 1.6161, Test Accuracy: 0.4120
Round 2/30, Test Loss: 1.3858, Test Accuracy: 0.5170
Round 3/30, Test Loss: 1.3084, Test Accuracy: 0.5470
Round 4/30, Test Loss: 1.2189, Test Accuracy: 0.5740
Round 5/30, Test Loss: 1.1466, Test Accuracy: 0.5950
Round 6/30, Test Loss: 1.0927, Test Accuracy: 0.6250
Round 7/30, Test Loss: 1.0541, Test Accuracy: 0.6280
Round 8/30, Test Loss: 1.0195, Test Accuracy: 0.6500
Round 9/30, Test Loss: 0.9882, Test Accuracy: 0.6680
Round 10/30, Test Loss: 0.9391, Test Accuracy: 0.6890
Round 11/30, Test Loss: 0.9317, Test Accuracy: 0.6950
Round 12/30, Test Loss: 0.9162, Test Accuracy: 0.7060
Round 13/30, Test Loss: 0.8962, Test Accuracy: 0.7080
Round 14/30, Test Loss: 0.8715, Test Accuracy: 0.7110
Round 15/30, Test Loss: 0.8619, Test Accuracy: 0.7150
Round 16/30, Test Loss: 0.8775, Test Accuracy: 0.7150
Round 17/30, Test Loss: 0.8579, Test Accuracy: 0.7180
Round 18/30, Te

## MobileNet_v2

In [12]:
architecture = 'mobilenet_v2'
federated_training_fedavg(architecture, num_clients, num_rounds)

Files already downloaded and verified
Files already downloaded and verified
Round 1/30, Test Loss: 2.0782, Test Accuracy: 0.2740
Round 2/30, Test Loss: 1.4924, Test Accuracy: 0.4880
Round 3/30, Test Loss: 1.2624, Test Accuracy: 0.5660
Round 4/30, Test Loss: 1.1682, Test Accuracy: 0.6110
Round 5/30, Test Loss: 1.1602, Test Accuracy: 0.6410
Round 6/30, Test Loss: 1.1652, Test Accuracy: 0.6470
Round 7/30, Test Loss: 1.1880, Test Accuracy: 0.6550
Round 8/30, Test Loss: 1.1858, Test Accuracy: 0.6680
Round 9/30, Test Loss: 1.2039, Test Accuracy: 0.6740
Round 10/30, Test Loss: 1.1871, Test Accuracy: 0.6850
Round 11/30, Test Loss: 1.2137, Test Accuracy: 0.6810
Round 12/30, Test Loss: 1.2100, Test Accuracy: 0.6900
Round 13/30, Test Loss: 1.2255, Test Accuracy: 0.6960
Round 14/30, Test Loss: 1.2185, Test Accuracy: 0.7090
Round 15/30, Test Loss: 1.2176, Test Accuracy: 0.7080
Round 16/30, Test Loss: 1.2349, Test Accuracy: 0.7040
Round 17/30, Test Loss: 1.2292, Test Accuracy: 0.7140
Round 18/30, Te

## ResNet18

In [13]:
architecture = 'resnet18' #'resnet18', 'mobilenet_v2', 'squeezenet' ou 'alexnet'
federated_training_fedavg(architecture, num_clients, num_rounds)

Files already downloaded and verified
Files already downloaded and verified
Round 1/30, Test Loss: 1.7441, Test Accuracy: 0.4050
Round 2/30, Test Loss: 1.2891, Test Accuracy: 0.5900
Round 3/30, Test Loss: 1.2596, Test Accuracy: 0.6290
Round 4/30, Test Loss: 1.1937, Test Accuracy: 0.6670
Round 5/30, Test Loss: 1.1597, Test Accuracy: 0.6770
Round 6/30, Test Loss: 1.1462, Test Accuracy: 0.6750
Round 7/30, Test Loss: 1.1632, Test Accuracy: 0.6850
Round 8/30, Test Loss: 1.1723, Test Accuracy: 0.6840
Round 9/30, Test Loss: 1.1585, Test Accuracy: 0.6870
Round 10/30, Test Loss: 1.1580, Test Accuracy: 0.6890
Round 11/30, Test Loss: 1.2336, Test Accuracy: 0.6900
Round 12/30, Test Loss: 1.1742, Test Accuracy: 0.7110
Round 13/30, Test Loss: 1.1878, Test Accuracy: 0.7150
Round 14/30, Test Loss: 1.2257, Test Accuracy: 0.7080
Round 15/30, Test Loss: 1.2135, Test Accuracy: 0.7150
Round 16/30, Test Loss: 1.2444, Test Accuracy: 0.7080
Round 17/30, Test Loss: 1.2354, Test Accuracy: 0.7200
Round 18/30, Te

## AlexNet

In [3]:
architecture = 'alexnet' #'resnet18', 'mobilenet_v2', 'squeezenet' ou 'alexnet'
federated_training_fedavg(architecture, num_clients, num_rounds)

Files already downloaded and verified
Files already downloaded and verified
Round 1/30, Test Loss: 2.2271, Test Accuracy: 0.1320
Round 2/30, Test Loss: 1.1507, Test Accuracy: 0.6170
Round 3/30, Test Loss: 0.8906, Test Accuracy: 0.7090
Round 4/30, Test Loss: 0.8720, Test Accuracy: 0.7530
Round 5/30, Test Loss: 0.8611, Test Accuracy: 0.7570
Round 6/30, Test Loss: 0.8153, Test Accuracy: 0.7760
Round 7/30, Test Loss: 0.7927, Test Accuracy: 0.7740
Round 8/30, Test Loss: 0.7631, Test Accuracy: 0.7750
Round 9/30, Test Loss: 0.7702, Test Accuracy: 0.7860
Round 10/30, Test Loss: 0.7586, Test Accuracy: 0.7940
Round 11/30, Test Loss: 0.7674, Test Accuracy: 0.7960
Round 12/30, Test Loss: 0.7554, Test Accuracy: 0.7890
Round 13/30, Test Loss: 0.7637, Test Accuracy: 0.8000
Round 14/30, Test Loss: 0.7582, Test Accuracy: 0.7970
Round 15/30, Test Loss: 0.7540, Test Accuracy: 0.7940
Round 16/30, Test Loss: 0.7777, Test Accuracy: 0.7970
Round 17/30, Test Loss: 0.7926, Test Accuracy: 0.8040
Round 18/30, Te